# Traits

## Defining Traits

* Traits are Rust’s take on interfaces or abstract base classes

In [2]:
#[derive(Debug, Copy, Clone)]
struct Point {
    x: i32,
    y: i32
}

trait Shape {
    fn area(&self) -> f64;

    fn get_coord(&self) -> Point;
    fn get_coord_mut(&mut self) -> &mut Point;

    fn move_by(&mut self, dx: i32, dy: i32) {
        let mut current_coord = self.get_coord_mut();
        current_coord.x += dx;
        current_coord.y += dy;
    }   
}

## Implementing Traits

* To implement a trait for a type, we use the `impl` keyword

In [3]:


#[derive(Debug, Copy, Clone)]
struct Circle {
    position: Point,
    radius: u32
}

#[derive(Debug, Copy, Clone)]
struct Rectangle {
    position: Point,
    width: u32,
    height: u32
}

impl Shape for Circle {
    // fn move_by(&mut self, dx: i32, dy: i32) {
    //     self.position.x += dx;
    //     self.position.y += dy;
    // }

    fn get_coord(&self) -> Point {
        self.position
    }

    fn get_coord_mut(&mut self) -> &mut Point {
        &mut self.position
    }

    fn area(&self) -> f64 {
        std::f64::consts::PI * (self.radius as f64) * (self.radius as f64)
    }
}

impl Shape for Rectangle {
    // fn move_by(&mut self, dx: i32, dy: i32) {
    //     self.position.x += dx;
    //     self.position.y += dy;
    // }

    fn get_coord(&self) -> Point {
        self.position
    }

    fn get_coord_mut(&mut self) -> &mut Point {
        &mut self.position
    }

    fn area(&self) -> f64 {
        (self.width * self.height) as f64
    }
}

In [4]:
let mut circle = Circle { position: Point { x: 0, y: 0 }, radius: 10 };
circle.move_by(10, 20);

println!("{:?}", circle);

Circle { position: Point { x: 10, y: 20 }, radius: 10 }


* We can implement many traits for a type:

In [5]:
trait Drawable {
    fn draw(&self);
}

impl Drawable for Circle {
    fn draw(&self) {
        println!("Drawing the circle at {:?} with radius: {}", self.position, self.radius);
    }
}

impl Drawable for Rectangle {
    fn draw(&self) {
          println!("Drawing the rectangle at {:?} with width: {} and height: {}", self.position, self.width, self.height);
  }
}

fn main() {
    let mut c = Circle { position: Point { x: 0, y: 0 }, radius: 10 };
    let mut r = Rectangle { position: Point { x: 0, y: 0 }, width: 10, height: 20 };

    c.draw();
    r.draw();

    c.move_by(10, 20);
    r.move_by(10, 20);

    println!("--- after move:");

    c.draw();
    r.draw();
}

main();

Drawing the circle at Point { x: 0, y: 0 } with radius: 10
Drawing the rectangle at Point { x: 0, y: 0 } with width: 10 and height: 20
--- after move:
Drawing the circle at Point { x: 10, y: 20 } with radius: 10
Drawing the rectangle at Point { x: 10, y: 20 } with width: 10 and height: 20


## Trait Bounds

* Trait bounds specify that a generic can be any type that implements a trait. 
* They work like a contracts at compile time

### impl Trait Syntax

* The `impl Trait` syntax is a shorthand for a longer form, called a trait bound

In [6]:
fn draw_shape(shape: &impl Drawable) {
    shape.draw()
}

In [7]:
let mut circle = Circle{position: Point{x: 10, y: 20}, radius: 5};
let mut rect = Rectangle{position: Point{x: 40, y: 50}, width: 10, height: 5};

draw_shape(&circle);
draw_shape(&rect);

Drawing the circle at Point { x: 10, y: 20 } with radius: 5
Drawing the rectangle at Point { x: 40, y: 50 } with width: 10 and height: 5


### Trait Bound Syntax

In [8]:
fn draw_shape<T: Drawable>(shape: &T) {
    shape.draw()
}

draw_shape(&circle);
draw_shape(&rect);


struct Sprite {
    position: Point,
    color: (u8, u8, u8),
}

fn draw_obj<T: std::fmt::Debug + Drawable>(obj: &T) {
    println!("Drawing the sprite {:?}", obj);
    obj.draw();
}

Drawing the circle at Point { x: 10, y: 20 } with radius: 5
Drawing the rectangle at Point { x: 40, y: 50 } with width: 10 and height: 5


### Many trait bounds

* We can specify multiple trait bounds on a generic type by using the `+` syntax

In [9]:
fn use_shape(shape: &mut (impl Shape + Drawable)) {
    shape.move_by(10, 15);
    shape.draw();
}

In [10]:
fn use_shape<T: Shape + Drawable>(shape: &mut T) {
    shape.move_by(10, 15);
    shape.draw();
}

* We can use a `where` clause to specify trait bounds on generic types

In [11]:
trait CustomShape: Shape + Drawable + std::fmt::Debug {
}

impl CustomShape for Circle {}
impl CustomShape for Rectangle {}

fn use_shapes<T, U>(shape1: &mut T, shape2: &mut U)
where 
    T: Shape + Drawable,
    U: CustomShape
{
    shape1.move_by(10, 15);
    shape1.draw();
    shape2.move_by(10, 15);
    shape2.draw();
    println!("{:?}", shape2);
}

In [27]:
use_shapes(&mut circle, &mut rect);
use_shapes(&mut rect, &mut circle);

Drawing the circle at Point { x: 40, y: 65 } with radius: 5
Drawing the rectangle at Point { x: 70, y: 95 } with width: 10 and height: 5
Rectangle { position: Point { x: 70, y: 95 }, width: 10, height: 5 }
Drawing the rectangle at Point { x: 80, y: 110 } with width: 10 and height: 5
Drawing the circle at Point { x: 50, y: 80 } with radius: 5
Circle { position: Point { x: 50, y: 80 }, radius: 5 }


### Returning Types that Implement Traits

In [13]:
fn create_drawable(id: &str) -> impl Drawable {
    match id {
        "circle" => Circle { position: Point { x: 0, y: 0 }, radius: 10 },
        //"rectangle" => Rectangle { position: Point { x: 0, y: 0 }, width: 10, height: 20 },
        _ => panic!("Unknown shape")
    }
}

let mut shape = create_drawable("circle");

Error: The variable `shape` has type `impl Drawable` which cannot be persisted.
You might be able to fix this by creating a `Box<dyn YourType>`. e.g.
let v: Box<dyn core::fmt::Debug> = Box::new(foo());
Alternatively, you can prevent evcxr from attempting to persist
the variable by wrapping your code in braces.

In [36]:
use std::rc::Rc;

fn create_drawable(id: &str) -> Box<dyn Drawable> {
    match id {
        "Circle" => Box::new(Circle{position: Point{x: 0, y: 0}, radius: 1}),
        "Rectangle" => Box::new(Rectangle{position: Point{x: 0, y: 0}, width: 2, height: 1}),
        _ => panic!("Unknown shape")
    }
}

let shape: Box<dyn Drawable> = create_drawable("Circle");
shape.draw();

Drawing the circle at Point { x: 0, y: 0 } with radius: 1


### Self in Traits

* `Self` is an alias for the type we’re implementing the trait for

In [37]:
trait Cloneable {
    fn clone(&self) -> Self; // the type of x.clone() is the same as the type of x, whatever that might be
}

* A trait that uses the `Self` type is incompatible with trait objects:

In [38]:
fn do_clone(prototype: &dyn Cloneable) -> Box<dyn Cloneable> {
    let cloned_object = prototype.clone();
    Box::new(cloned_object)
}

Error: the trait `Cloneable` cannot be made into an object

Error: the trait `Cloneable` cannot be made into an object

Error: the size for values of type `dyn Cloneable` cannot be known at compilation time

Error: the trait `Cloneable` cannot be made into an object

Error: the size for values of type `dyn Cloneable` cannot be known at compilation time

Error: the trait `Cloneable` cannot be made into an object

## Trait Objects

* A trait object allows to implement dynamic polymorphism in Rust

```c++
class Shape {
public:
    virtual void move_by(int dx, int dy) = 0;
    virtual double area() const = 0;
    virtual ~Shape() = default;
};

class Rectangle : public Shape {
public:
    void move_by(int dx, int dy) override {
        position.x += dx;
        position.y += dy;
    }

    double area() const override {
        return width * height;
    }

    Point position;
    int width;
    int height;
};

class Circle : public Shape {
public:
    void move_by(int dx, int dy) override {
        position.x += dx;
        position.y += dy;
    }

    double area() const override {
        return 3.14159 * radius * radius;
    }

    Point position;
    int radius;
};

int main() {
    Circle c;
    Rectangle r;

    Shape* shp = &r;
    shp->move_by(10, 20);

    std::vector<std::unique_ptr<Shape>> shapes;
    shapes.push_back(std::make_unique<Circle>());
    shapes.push_back(std::make_unique<Rectangle>());
}

```

In [56]:
mod explain {
    struct Point {
        x: i32,
        y: i32
    }

    trait Shape {
        fn move_by(&mut self, dx: i32, dy: i32);
        fn area(&self) -> f64;
    }

    struct Rectangle {
        position: Point,
        width: u32,
        height: u32
    }

    impl Shape for Rectangle {
        fn move_by(&mut self, dx: i32, dy: i32) {
            self.position.x += dx;
            self.position.y += dy;
        }

        fn area(&self) -> f64 {
            (self.width * self.height) as f64
        }
    }

    struct Circle {
        position: Point,
        radius: u32
    }

    impl Shape for Circle {
        fn move_by(&mut self, dx: i32, dy: i32) {
            self.position.x += dx;
            self.position.y += dy;
        }

        fn area(&self) -> f64 {
            std::f64::consts::PI * (self.radius as f64) * (self.radius as f64)
        }
    }
}

fn static_polymorphism() {
    fn use_shape<T: Shape>(shape: &mut T) {
        shape.move_by(10, 15);
        println!("Area: {}", shape.area());
    }

    let mut rect = Rectangle { position: Point { x: 0, y: 0 }, width: 10, height: 20 };
    let mut circle = Circle { position: Point { x: 0, y: 0 }, radius: 10 };

    use_shape(&mut rect);
    use_shape(&mut circle);
}

fn dynamic_polymorphism() {
    fn use_shape(shape: &mut dyn Shape) {
        shape.move_by(10, 15);
        println!("Area: {}", shape.area());
    }

    let mut rect: Rectangle = Rectangle { position: Point { x: 0, y: 0 }, width: 10, height: 20 };
    let mut circle: Circle = Circle { position: Point { x: 0, y: 0 }, radius: 10 };

    use_shape(&mut rect);
    use_shape(&mut circle);    

    println!("\nVector of shapes:");

    let mut shapes: Vec<Box<dyn Shape>> = Vec::new();
    shapes.push(Box::new(rect));
    shapes.push(Box::new(circle));   

    for shape in shapes.iter_mut() {
        // shape.move_by(10, 15);
        // println!("Area: {}", shape.area());
        use_shape(shape.as_mut());
    }
}

fn main() {
    static_polymorphism();
    println!("---------------------------------");
    dynamic_polymorphism();
}

main();

Area: 200
Area: 314.1592653589793
---------------------------------
Area: 200
Area: 314.1592653589793

Vector of shapes:
Area: 200
Area: 314.1592653589793


### References to Trait

* A reference to trait type, like `Shape` is called **trait object**. Like any other reference, a trait object points to some value, it has a lifetime, and it can be either mut or shared.
* In memory, a trait object is a **fat pointer** consisting of a pointer to the value, plus a pointer to a table representing that value’s type (virtual table).



In [68]:
fn main() {
    let mut shape: &dyn Shape;
    
    let circle = Circle{position: Point{x: 10, y: 20}, radius: 5};
    shape = &circle;
    println!("Shape area: {:.2}", shape.area());

    shape = &Rectangle{position: Point{x: 10, y: 20}, width: 10, height: 20};
    println!("Shape area: {:.2}", shape.area());
}

main();

Shape area: 78.54
Shape area: 200.00


### Boxes to Trait

* We can also use a `Box` to store a trait object
* A `Box<dyn Trait>` is a trait object that points to a value on the heap

In [69]:
let box_circle: Box<Circle> = Box::new(Circle{position: Point{x: 10, y: 20}, radius: 5});
let mut shape: Box<dyn Shape> = box_circle; // OK - Box<dyn Shape> is trait object
println!("Shape area: {:.2}", shape.area());

shape = Box::new(Rectangle{position: Point{x: 10, y: 20}, width: 10, height: 20});
println!("Shape area: {:.2}", shape.area());

Shape area: 78.54
Shape area: 200.00


## Dynamic Polymorphism in Rust

In [70]:
#[derive(Debug)]
struct Square {
    position: Point,
    side: u32
}   

impl Shape for Square {

    fn move_by(&mut self, dx: i32, dy: i32) {
        self.position.x += dx;
        self.position.y += dy;
    }

    fn area(&self) -> f64 {
        (self.side * self.side) as f64
    }
}

impl Drawable for Square {
    fn draw(&self) {
        println!("Drawing the square at {:?} with side: {}", self.position, self.side);
    }
}

fn main() {
    let mut shapes: Vec<Box<dyn Shape>> = vec![
        Box::new(Circle{position: Point{x: 10, y: 20}, radius: 5}),
        Box::new(Rectangle{position: Point{x: 40, y: 50}, width: 10, height: 5}),
        Box::new(Square{position: Point{x: 70, y: 80}, side: 7})
    ];

    for shape in shapes.iter_mut() {
        shape.move_by(5, 5);
        println!("Shape area: {:.2}", shape.area());
    }
}

main();

Shape area: 78.54
Shape area: 50.00
Shape area: 49.00


### Combining Traits

In [57]:
trait DrawableShape: Shape + Drawable {} // Trait that combines Shape and Drawable

impl<T> DrawableShape for T where T: Shape + Drawable {} // Implement DrawableShape for any type that implements Shape and Drawable

fn use_shape(shape: &mut dyn DrawableShape) {
    shape.draw();
    shape.move_by(10, 15);
    shape.draw();
}

fn main() {
    let mut circle = Circle{position: Point{x: 10, y: 20}, radius: 5};
    use_shape(&mut circle);

    let mut rect = Rectangle{position: Point{x: 40, y: 50}, width: 10, height: 5};
    use_shape(&mut rect);
}

main();

Drawing the circle at Point { x: 10, y: 20 } with radius: 5
Drawing the circle at Point { x: 20, y: 35 } with radius: 5
Drawing the rectangle at Point { x: 40, y: 50 } with width: 10 and height: 5
Drawing the rectangle at Point { x: 50, y: 65 } with width: 10 and height: 5


## Orphan Rules

* The orphan rules prevent us from implementing a foreign trait on a foreign type
* You are allowed to implement any trait on any type, as long as either the trait or the type is local to your crate

In [59]:
trait IsEmoji {
    fn is_emoji(&self) -> bool;
}

impl IsEmoji for char {
    fn is_emoji(&self) -> bool {
        let c = *self as u32;
        (c >= 0x1F600 && c <= 0x1F64F) || (c >= 0x1F300 && c <= 0x1F5FF) || (c >= 0x1F680 && c <= 0x1F6FF) || (c >= 0x1F900 && c <= 0x1F9FF)
    }
}

fn main() {
    let c = '😀';
    println!("Is '{}' an emoji? {}", c, c.is_emoji());
    println!("Is '{}' an emoji? {}", '@', '@'.is_emoji());
}

main();

Is '😀' an emoji? true


Is '@' an emoji? false


* `IsEmoji` is an *extension trait* - adds a method to an existing type

* We can use generic `impl` block to add an extension trait to a whole family of types

In [62]:
use std::fmt::Debug;

trait DrawInDebug {
    fn draw_dbg(&self);
}

impl<T: Debug + Drawable> DrawInDebug for T {
    fn draw_dbg(&self) {
        println!("---- Drawing a {:?}", self);
    }
}

In [63]:
circle.draw_dbg();
rect.draw_dbg();

---- Drawing a Circle { position: Point { x: 50, y: 80 }, radius: 5 }
---- Drawing a Rectangle { position: Point { x: 80, y: 110 }, width: 10, height: 5 }


## Associated Types

* Associated types are a way to associate a type placeholder with a trait
* Each type that implements the trait can specify its own concrete type for the associated type

In [19]:
trait Iterator {
    type Item; // associated type
    
    fn next(&mut self) -> Option<Self::Item>;

    // default implementation
    fn for_each(&mut self, mut f: impl FnMut(Self::Item)) {
        while let Some(value) = self.next() {
            f(value);
        }
    }
}

struct Counter {
    value: u32,
    sentinel: u32
}

impl Counter {
    fn new(sentinel: u32) -> Counter {
        Counter{value: 0, sentinel}
    }
}

impl Iterator for Counter {
    type Item = u32; // definition of associated type for Counter

    fn next(&mut self) -> Option<Self::Item> {
        if self.value < self.sentinel {
            let result = self.value;
            self.value += 1;
            Some(result)
        } else {
            None
        }
    }
}

fn main() {
    let mut counter = Counter::new(5);

    counter.for_each(|value| println!("Printing: {}", value));

    println!("---");

    let mut counter = Counter::new(4);

    while let Some(value) = counter.next() {
        println!("{}", value);
    }
}

main();

Printing: 0
Printing: 1
Printing: 2
Printing: 3
Printing: 4
---
0
1
2
3


* Generic code can use associated types:

In [20]:
fn collect_into_vector<I: Iterator>(mut iter: I) -> Vec<I::Item> // I::Item is the associated type of the Iterator
{
    let mut results = Vec::new();

    while let Some(value) = iter.next() {
        results.push(value);
    }

    results
}

fn main() {    
    let numbers = collect_into_vector(Counter::new(5));
    println!("Collected numbers: {:?}", numbers);
}

main();

Collected numbers: [0, 1, 2, 3, 4]


## Subtraits

* A trait can be an extension of another trait

In [21]:
trait BidirectionalIterator: Iterator {
    fn prev(&mut self) -> Option<Self::Item>;
}

impl BidirectionalIterator for Counter {
    fn prev(&mut self) -> Option<Self::Item> {
        if self.value > 0 {
            let result = self.value;
            self.value -= 1;
            Some(self.value)
        } else {
            None
        }
    }
}

fn main() {
    let mut counter = Counter::new(5);

    println!("Forward iteration:");
    while let Some(value) = counter.next() {
        println!("Counter value: {}", value);
    }

    println!("\nBackward iteration:");
    while let Some(value) = counter.prev() {
        println!("Counter value: {}", value);
    }
}

main();

Forward iteration:
Counter value: 0
Counter value: 1
Counter value: 2
Counter value: 3
Counter value: 4

Backward iteration:
Counter value: 4
Counter value: 3
Counter value: 2
Counter value: 1
Counter value: 0


## Static Methods

* Traits can have static methods

In [22]:
trait StringSet {
    /// Creates a new, empty set. - static method
    fn new() -> Self;

    /// Creates a set from a slice of strings. - static method
    fn from_slice(values: &[&str]) -> Self;

    /// Adds a string to the set.
    fn add(&mut self, value: &str);

    /// Removes a string from the set.
    fn contains(&self, value: &str) -> bool;
}

struct HashSet {
    data: std::collections::HashSet<String>
}

impl StringSet for HashSet {
    fn new() -> Self {
        HashSet{data: std::collections::HashSet::new()}
    }

    fn from_slice(values: &[&str]) -> Self {
        let mut set = HashSet::new();
        for value in values {
            set.add(value);
        }
        set
    }

    fn add(&mut self, value: &str) {
        self.data.insert(value.to_string());
    }

    fn contains(&self, value: &str) -> bool {
        self.data.contains(value)
    }
}

fn main() {
    println!("Creating set:");
    let mut set = HashSet::new();
    set.add("apple");
    set.add("banana");
    set.add("orange");

    println!("Set contains 'apple': {}", set.contains("apple"));
    println!("Set contains 'grape': {}", set.contains("grape"));

    println!("\nCreating set from slice:");

    let set = HashSet::from_slice(&["apple", "banana", "orange"]);
    println!("Set contains 'apple': {}", set.contains("apple"));
    println!("Set contains 'grape': {}", set.contains("grape"));
}

main();

Creating set:
Set contains 'apple': true
Set contains 'grape': false

Creating set from slice:
Set contains 'apple': true
Set contains 'grape': false


## Fully qualified methods

In [24]:
let mut circle = Circle{position: Point{x: 10, y: 20}, radius: 5};
let mut rectangle = Rectangle{position: Point{x: 40, y: 50}, width: 10, height: 5};

circle.draw();
Shape::move_by(&mut circle, 10, 15);
Drawable::draw(&circle);

<Rectangle as Drawable>::draw(&rectangle);
rectangle.move_by(10, 15);
Drawable::draw(&rectangle);

Drawing the circle at Point { x: 10, y: 20 } with radius: 5
Drawing the circle at Point { x: 20, y: 35 } with radius: 5
Drawing the rectangle at Point { x: 40, y: 50 } with width: 10 and height: 5
Drawing the rectangle at Point { x: 50, y: 65 } with width: 10 and height: 5


* Fully qualified method calls are used:
  * to disambiguate between methods with the same name

## Reverse-Engineering Bounds

* First attempt

In [35]:
fn dot<N>(v1: &[N], v2: &[N]) -> N 
    where N:  Default,
          N:  std::ops::Mul<Output=N> + std::ops::Add<Output=N>,
          N:  Copy
{
    let mut total: N = N::default();
    for i in 0 .. v1.len() {
        total = total + v1[i] * v2[i];
    }
    total
}

In [81]:
use std::ops::{Add, Mul};

fn dot<N: Default + Mul<Output=N> + Add<Output=N> + Copy>(v1: &[N], v2: &[N]) -> N 
{
    let mut total: N = N::default();
    for i in 0 .. v1.len() {
        total = total + v1[i] * v2[i];
    }
    total
}

# Generics

## Generic Function

In [40]:
fn min<T: PartialOrd>(a: T, b: T) -> T {
    if a < b {
        a
    } else {
        b
    }
}

In [42]:
min(1, 2)

1

In [43]:
min(3.41, 2.1)

2.1

In [44]:
min("hello", "world")

"hello"

## Generic Structure

In [85]:
struct Stack<T> {
    items: Vec<T>
}

impl<T> Stack<T> {
    fn push(&mut self, item: T) {
        self.items.push(item);
    }

    fn pop(&mut self) -> Option<T> {
        self.items.pop()
    }
}

fn main() {
    let mut stack = Stack{items: Vec::new()};
    stack.push(1);
    stack.push(2);
    stack.push(3);

    while let Some(item) = stack.pop() {
        println!("Popped: {}", item);
    }
}

main();

Popped: 3
Popped: 2
Popped: 1


## Generic Traits

In [86]:
mod explain {
    /// std::ops::Mul, the trait for types that support `*`.
    pub trait Mul<RHS=Self> {
        /// The resulting type after applying the `*` operator
        type Output;

        /// The method for the `*` operator
        fn mul(self, rhs: RHS) -> Self::Output;
    }
}

In [ ]:
#[derive(Debug)]
struct Complex {
    real: f64,
    imag: f64
}

use std::ops::Mul;

impl Mul for Complex {
    type Output = Complex;

    fn mul(self, rhs: Complex) -> Complex {
        Complex {
            real: self.real * rhs.real - self.imag * rhs.imag,
            imag: self.real * rhs.imag + self.imag * rhs.real
        }
    }
}

fn main() {
    let a = Complex{real: 1.0, imag: 2.0};
    let b = Complex{real: 3.0, imag: 4.0};

    let c = a * b;
    println!("Result: {} + {}i", c.real, c.imag);
}

main();

Error: method `mul_` is not a member of trait `Mul`

Error: not all trait items implemented, missing: `mul`

Error: unused variable: `result`